[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ourownstory/neural_prophet/blob/master/example_notebooks/autoregression_yosemite_temps.ipynb)

# Hyperparameter optimization with Ray Tune

We introduce the module for hyperparameter otpimization with Ray Tune.

It supports automatic tuning, with predefined by us hyperparameter sets, as well as manual tuning with user-provided configuration of the parameters.

Firstly, we will show how it works with NP model in automated mode.

In [ ]:
# install NeuralProphet from our repository
!pip install git+https://github.com/adasegroup/neural_prophet.git # may take a while
!pip install tensorboardX

In [13]:
import pandas as pd
import numpy as np
from neuralprophet import NeuralProphet

In [14]:
from neuralprophet.hyperparameter_tuner import tune_hyperparameters

In [15]:
if 'google.colab' in str(get_ipython()):
    data_location = "https://raw.githubusercontent.com/adasegroup/neural_prophet/master/"
else:
    data_location = "../"

df = pd.read_csv(data_location + "example_data/yosemite_temps.csv")
df.head(3)

,ds,y
0,2017-05-01 00:00:00,27.8
1,2017-05-01 00:05:00,27.0
2,2017-05-01 00:10:00,26.8


In [17]:
freq = '5min'
best_params, results_df = tune_hyperparameters('NP',
                                               df,
                                               freq)

(raylet) /Users/polina/.conda/envs/neural_prophet/lib/python3.7/site-packages/ray/autoscaler/_private/cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
(raylet)   "update your install command.", FutureWarning)
(raylet) /Users/polina/.conda/envs/neural_prophet/lib/python3.7/site-packages/ray/autoscaler/_private/cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
(raylet)   "update your install command.", FutureWarning)
(raylet) /Users/polina/.conda/envs/neural_prophet/lib/python3.7/site-packages/ray/autoscaler/_private/cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable 

(pid=18489) {'growth': 'off', 'n_changepoints': 100, 'changepoints_range': 0.5, 'trend_reg': 10.0, 'yearly_seasonality': False, 'weekly_seasonality': True, 'daily_seasonality': False, 'seasonality_mode': 'additive', 'seasonality_reg': 1.0, 'n_lags': 10, 'd_hidden': 128, 'num_hidden_layers': 2, 'ar_sparsity': 0.1, 'learning_rate': 0.00025794771866301903, 'loss_func': 'MSE', 'normalize': 'standardize'}
(pid=18492) {'growth': 'linear', 'n_changepoints': 100, 'changepoints_range': 0.5, 'trend_reg': 0.0, 'yearly_seasonality': False, 'weekly_seasonality': False, 'daily_seasonality': True, 'seasonality_mode': 'multiplicative', 'seasonality_reg': 0.5, 'n_lags': 10, 'd_hidden': 8, 'num_hidden_layers': 2, 'ar_sparsity': 0.3, 'learning_rate': 0.0002926262239414852, 'loss_func': 'MSE', 'normalize': 'auto'}
(pid=18491) {'growth': 'off', 'n_changepoints': 100, 'changepoints_range': 0.5, 'trend_reg': 0.0, 'yearly_seasonality': False, 'weekly_seasonality': False, 'daily_seasonality': False, 'seasonali

  0%|          | 0/100 [00:00<?, ?it/s]GPU available: False, used: False
(pid=18492) TPU available: False, using: 0 TPU cores
(pid=18492) GPU available: False, used: False
(pid=18492) TPU available: False, using: 0 TPU cores
(pid=18492) 
(pid=18492)   | Name          | Type          | Params
(pid=18492) ------------------------------------------------
(pid=18492) 0 | season_params | ParameterDict | 12    
(pid=18492) 1 | ar_net        | ModuleList    | 168   
(pid=18492) 2 | loss_func     | MSELoss       | 0     
(pid=18492) ------------------------------------------------
(pid=18492) 283       Trainable params
(pid=18492) 0         Non-trainable params
(pid=18492) 283       Total params
(pid=18492) 0.001     Total estimated model params size (MB)
(pid=18492) WARNING - (py.warnings._showwarnmsg) - /Users/polina/.conda/envs/neural_prophet/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:68: UserWarning: The dataloader, train dataloader, does not have many workers w

(pid=18486) {'growth': 'off', 'n_changepoints': 5, 'changepoints_range': 0.9, 'trend_reg': 10.0, 'yearly_seasonality': True, 'weekly_seasonality': False, 'daily_seasonality': True, 'seasonality_mode': 'multiplicative', 'seasonality_reg': 0.5, 'n_lags': 10, 'd_hidden': 64, 'num_hidden_layers': 8, 'ar_sparsity': 0.8, 'learning_rate': 0.05709530464716042, 'loss_func': 'Huber', 'normalize': 'soft'}
(pid=18487) {'growth': 'off', 'n_changepoints': 5, 'changepoints_range': 0.8, 'trend_reg': 0.0, 'yearly_seasonality': True, 'weekly_seasonality': False, 'daily_seasonality': True, 'seasonality_mode': 'multiplicative', 'seasonality_reg': 1.0, 'n_lags': 10, 'd_hidden': 128, 'num_hidden_layers': 16, 'ar_sparsity': 0.3, 'learning_rate': 0.004413941508220393, 'loss_func': 'Huber', 'normalize': 'soft'}
(pid=18493) {'growth': 'linear', 'n_changepoints': 10, 'changepoints_range': 0.5, 'trend_reg': 0.5, 'yearly_seasonality': True, 'weekly_seasonality': True, 'daily_seasonality': True, 'seasonality_mode':

  0%|          | 0/100 [00:00<?, ?it/s]
(raylet) /Users/polina/.conda/envs/neural_prophet/lib/python3.7/site-packages/ray/autoscaler/_private/cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
(raylet)   "update your install command.", FutureWarning)
(pid=18532) INFO - (NP.config.__post_init__) - Note: Trend changepoint regularization is experimental.
(pid=18532) INFO - (NP.config.__post_init__) - Note: Fourier-based seasonality regularization is experimental.
(pid=18532) INFO - (NP.forecaster._handle_missing_data) - 12 NaN values in column y were auto-imputed.
(pid=18532) INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 64
  0%|          | 0/100 [00:00<?, ?it/s]GPU available: False, used: False
(pid=18532) TPU available: False, using: 0 TPU cores
(pid=18532) GPU available: False, used: False
(pid=18532) TPU

...

(pid=18695) {'growth': 'off', 'n_changepoints': 100, 'changepoints_range': 0.9, 'trend_reg': 0.5, 'yearly_seasonality': True, 'weekly_seasonality': True, 'daily_seasonality': False, 'seasonality_mode': 'multiplicative', 'seasonality_reg': 0.5, 'n_lags': 30, 'd_hidden': 64, 'num_hidden_layers': 2, 'ar_sparsity': 0.3, 'learning_rate': 0.0021617056005691173, 'loss_func': 'MSE', 'normalize': 'off'}


  0%|          | 0/100 [00:00<?, ?it/s]
(raylet) /Users/polina/.conda/envs/neural_prophet/lib/python3.7/site-packages/ray/autoscaler/_private/cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
(raylet)   "update your install command.", FutureWarning)
(pid=18698) INFO - (NP.config.__post_init__) - Note: Trend changepoint regularization is experimental.
(pid=18698) INFO - (NP.config.__post_init__) - Note: Fourier-based seasonality regularization is experimental.
(pid=18698) INFO - (NP.forecaster._handle_missing_data) - 12 NaN values in column y were auto-imputed.
(pid=18698) INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 64
  0%|          | 0/100 [00:00<?, ?it/s]GPU available: False, used: False
(pid=18698) TPU available: False, using: 0 TPU cores
(pid=18698) GPU available: False, used: False
(pid=18698) TPU

(pid=18698) {'growth': 'linear', 'n_changepoints': 5, 'changepoints_range': 0.8, 'trend_reg': 10.0, 'yearly_seasonality': True, 'weekly_seasonality': True, 'daily_seasonality': False, 'seasonality_mode': 'multiplicative', 'seasonality_reg': 0.5, 'n_lags': 30, 'd_hidden': 128, 'num_hidden_layers': 2, 'ar_sparsity': 0.3, 'learning_rate': 0.023518041055053677, 'loss_func': 'Huber', 'normalize': 'standardize'}


  0%|          | 0/100 [00:00<?, ?it/s]


Best hyperparameters found were:  {'growth': 'off', 'n_changepoints': 100, 'changepoints_range': 0.9, 'trend_reg': 1.0, 'yearly_seasonality': False, 'weekly_seasonality': False, 'daily_seasonality': False, 'seasonality_mode': 'multiplicative', 'seasonality_reg': 0.5, 'n_lags': 30, 'd_hidden': 8, 'num_hidden_layers': 8, 'ar_sparsity': 0.1, 'learning_rate': 0.016032870001225212, 'loss_func': 'Huber', 'normalize': 'minmax'}


This function by default outputs the dictionary of best hyperparameters chosen. It additionally will output the dataframe with detailed result of each trial if return_results is set to True.

In [16]:
results_df[['config.growth', 'config.n_changepoints', 'config.changepoints_range',
       'config.trend_reg', 'config.yearly_seasonality',
       'config.weekly_seasonality', 'config.daily_seasonality',
       'config.seasonality_mode', 'config.seasonality_reg', 'config.n_lags',
       'config.d_hidden', 'config.num_hidden_layers', 'config.ar_sparsity',
       'config.learning_rate', 'config.loss_func', 'config.normalize']]

,config.growth,config.n_changepoints,config.changepoints_range,config.trend_reg,config.yearly_seasonality,config.weekly_seasonality,config.daily_seasonality,config.seasonality_mode,config.seasonality_reg,config.n_lags,config.d_hidden,config.num_hidden_layers,config.ar_sparsity,config.learning_rate,config.loss_func,config.normalize
trial_id,,,,,,,,,,,,,,,,
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ecf09_00001,linear,10.0,0.9,0.5,True,True,True,additive,0.0,10.0,128.0,16.0,0.1,0.000340,Huber,auto
ecf09_00002,off,5.0,0.8,0.0,False,False,False,multiplicative,0.5,100.0,128.0,8.0,0.3,0.004222,MSE,off
ecf09_00003,linear,10.0,0.8,10.0,False,True,True,additive,0.0,100.0,64.0,8.0,0.3,0.001155,MSE,off
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ecf09_00007,linear,5.0,0.8,0.5,True,False,True,additive,10.0,30.0,8.0,16.0,0.3,0.098468,Huber,standardize
ecf09_00008,linear,10.0,0.5,0.0,True,False,True,additive,10.0,30.0,8.0,8.0,0.8,0.031956,MSE,auto


In [7]:
best_params

{'growth': 'off',
 'n_changepoints': 100,
 'changepoints_range': 0.8,
 'trend_reg': 0.0,
 'yearly_seasonality': False,
 'weekly_seasonality': False,
 'daily_seasonality': False,
 'seasonality_mode': 'additive',
 'seasonality_reg': 0.5,
 'n_lags': 100,
 'd_hidden': 8,
 'num_hidden_layers': 2,
 'ar_sparsity': 0.8,
 'learning_rate': 0.010444235692186717,
 'loss_func': 'Huber',
 'normalize': 'minmax'}

This dictionary can further be used in initialization of NeuralProphet model.

This function has also additional parameters:
- **num_epochs**: Max possible number of epochs to train each model.
- **num_samples**: Number of samples from hyperparameter spaces to check.
- **resources_per_trial**: Resources per trial setting for ray.tune.run, {'cpu': 1, 'gpu': 2} for example



## Manual mode

In case of manual mode, a user must provide a config dictionary with hyperparameter spaces in compatability with Ray Tune api.

We provide a minimal example below, for more information on Search Spaces withit this link https://docs.ray.io/en/master/tune/api_docs/search_space.html?highlight=tune.choice

In [5]:
from ray import tune

config = {'n_lags': tune.grid_search([10, 20, 30]),
          'learning_rate': tune.loguniform(1e-4, 1e-1),
          'num_hidden_layers': tune.choice([2, 8, 16])}

In [6]:
freq = '5min'
best_params, results_df = tune_hyperparameters('NP', 
                                               df,
                                               freq, 
                                               mode = 'manual',
                                              config = config)

2021-05-11 00:14:32,768	INFO services.py:1269 -- View the Ray dashboard at http://127.0.0.1:8265
2021-05-11 00:14:34,595	WARNING function_runner.py:545 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.
(pid=1261) INFO - (NP.forecaster._handle_missing_data) - 12 NaN values in column y were auto-imputed.
(pid=1261) INFO - (NP.utils.set_auto_seasonalities) - Disabling yearly seasonality. Run NeuralProphet with yearly_seasonality=True to override this.
(pid=1261) INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 64
  0%|          | 0/100 [00:00<?, ?it/s]GPU available: False, used: False
(pid=1261) TPU available: False, using: 0 TPU cores
(pid=1261) GPU available: False, used: False
(pid=1261) TPU available: False, using: 0 TPU cores
(pid=1261) 
(pid=1261)   | Name          | Type          | Params
(pid=126

(pid=1261) {'n_lags': 10, 'learning_rate': 0.00015556502370375923, 'num_hidden_layers': 8}


(pid=1265) INFO - (NP.forecaster._handle_missing_data) - 12 NaN values in column y were auto-imputed.
(pid=1265) INFO - (NP.utils.set_auto_seasonalities) - Disabling yearly seasonality. Run NeuralProphet with yearly_seasonality=True to override this.
(pid=1265) INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 64
(pid=1264) INFO - (NP.forecaster._handle_missing_data) - 12 NaN values in column y were auto-imputed.
(pid=1267) INFO - (NP.forecaster._handle_missing_data) - 12 NaN values in column y were auto-imputed.
(pid=1267) INFO - (NP.utils.set_auto_seasonalities) - Disabling yearly seasonality. Run NeuralProphet with yearly_seasonality=True to override this.
(pid=1267) INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 64
(pid=1264) INFO - (NP.utils.set_auto_seasonalities) - Disabling yearly seasonality. Run NeuralProphet with yearly_seasonality=True to override this.
(pid=1264) INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 64
(pid=1260)

(pid=1265) {'n_lags': 20, 'learning_rate': 0.0009797678796626623, 'num_hidden_layers': 2}
(pid=1260) {'n_lags': 30, 'learning_rate': 0.02355130060756577, 'num_hidden_layers': 8}
(pid=1264) {'n_lags': 20, 'learning_rate': 0.020016476301778063, 'num_hidden_layers': 2}
(pid=1267) {'n_lags': 30, 'learning_rate': 0.012420692727338386, 'num_hidden_layers': 2}
(pid=1263) {'n_lags': 20, 'learning_rate': 0.0017272300024959082, 'num_hidden_layers': 8}
(pid=1262) {'n_lags': 10, 'learning_rate': 0.04563978294862115, 'num_hidden_layers': 16}
(pid=1266) {'n_lags': 10, 'learning_rate': 0.038388510686423835, 'num_hidden_layers': 8}


  0%|          | 0/100 [00:00<?, ?it/s]GPU available: False, used: False
(pid=1266) TPU available: False, using: 0 TPU cores
(pid=1266) GPU available: False, used: False
(pid=1266) TPU available: False, using: 0 TPU cores
(pid=1266) 
(pid=1266)   | Name          | Type          | Params
(pid=1266) ------------------------------------------------
(pid=1266) 0 | season_params | ParameterDict | 18    
(pid=1266) 1 | ar_net        | ModuleList    | 1.1 K 
(pid=1266) 2 | loss_func     | SmoothL1Loss  | 0     
(pid=1266) ------------------------------------------------
(pid=1266) 1.1 K     Trainable params
(pid=1266) 0         Non-trainable params
(pid=1266) 1.1 K     Total params
(pid=1266) 0.004     Total estimated model params size (MB)
(pid=1266) WARNING - (py.warnings._showwarnmsg) - /Users/polina/.conda/envs/neural_prophet/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:68: UserWarning: The dataloader, train dataloader, does not have many workers which may be a b

...

  0%|          | 0/100 [00:00<?, ?it/s]
(pid=1784) INFO - (NP.forecaster._handle_missing_data) - 12 NaN values in column y were auto-imputed.
(pid=1784) INFO - (NP.utils.set_auto_seasonalities) - Disabling yearly seasonality. Run NeuralProphet with yearly_seasonality=True to override this.
(pid=1784) INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 64
  0%|          | 0/100 [00:00<?, ?it/s]GPU available: False, used: False
(pid=1784) TPU available: False, using: 0 TPU cores
(pid=1784) GPU available: False, used: False
(pid=1784) TPU available: False, using: 0 TPU cores
(pid=1784) 
(pid=1784)   | Name          | Type          | Params
(pid=1784) ------------------------------------------------
(pid=1784) 0 | season_params | ParameterDict | 18    
(pid=1784) 1 | ar_net        | ModuleList    | 7.4 K 
(pid=1784) 2 | loss_func     | SmoothL1Loss  | 0     
(pid=1784) ------------------------------------------------
(pid=1784) 7.4 K     Trainable params
(pid=1784) 0         N

(pid=1784) {'n_lags': 20, 'learning_rate': 0.00022418192183492564, 'num_hidden_layers': 16}


  0%|          | 0/100 [00:00<?, ?it/s]
(raylet) /Users/polina/.conda/envs/neural_prophet/lib/python3.7/site-packages/ray/autoscaler/_private/cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
(raylet)   "update your install command.", FutureWarning)
(pid=1790) INFO - (NP.forecaster._handle_missing_data) - 12 NaN values in column y were auto-imputed.
(pid=1790) INFO - (NP.utils.set_auto_seasonalities) - Disabling yearly seasonality. Run NeuralProphet with yearly_seasonality=True to override this.
(pid=1790) INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 64
  0%|          | 0/100 [00:00<?, ?it/s]GPU available: False, used: False
(pid=1790) TPU available: False, using: 0 TPU cores
(pid=1790) GPU available: False, used: False
(pid=1790) TPU available: False, using: 0 TPU cores
(pid=1790) 
(pid=1790)   | Name 

(pid=1790) {'n_lags': 30, 'learning_rate': 0.005619434333669411, 'num_hidden_layers': 8}


  0%|          | 0/100 [00:00<?, ?it/s]


In [7]:
results_df

,loss,time_this_iter_s,done,timesteps_total,episodes_total,training_iteration,experiment_id,date,timestamp,time_total_s,pid,hostname,node_ip,time_since_restore,timesteps_since_restore,iterations_since_restore,experiment_tag,config.n_lags,config.learning_rate,config.num_hidden_layers
trial_id,,,,,,,,,,,,,,,,,,,,
b82cf_00000,1.084538,1.495492,True,None,None,10,fe661eab9cdb4fbf9eb0e7b57f0ae02a,2021-05-11_00-14-56,1620681296,18.115594,1261,MacBook-Pro-Polina.local,192.168.0.7,18.115594,0,10,"0_learning_rate=0.00015557,n_lags=10,num_hidde...",10,0.000156,8
b82cf_00001,0.579620,1.627965,True,None,None,10,2611934321574e2ea23e7ccf6ac19e3c,2021-05-11_00-14-56,1620681296,17.901918,1263,MacBook-Pro-Polina.local,192.168.0.7,17.901918,0,10,"1_learning_rate=0.0017272,n_lags=20,num_hidden...",20,0.001727,8
b82cf_00002,0.000066,1.588995,True,None,None,100,49432d66f8a748eb8105909f268b6657,2021-05-11_00-16-56,1620681416,137.877947,1267,MacBook-Pro-Polina.local,192.168.0.7,137.877947,0,100,"2_learning_rate=0.012421,n_lags=30,num_hidden_...",30,0.012421,2
b82cf_00003,0.000032,2.032028,True,None,None,100,bde5cd3ed5df4f2fab4abfb03e97253f,2021-05-11_00-18-15,1620681495,217.408729,1262,MacBook-Pro-Polina.local,192.168.0.7,217.408729,0,100,"3_learning_rate=0.04564,n_lags=10,num_hidden_l...",10,0.045640,16
b82cf_00004,0.429556,1.509781,True,None,None,100,4582d742f14846e99003f8a7596e9083,2021-05-11_00-16-53,1620681413,135.652835,1265,MacBook-Pro-Polina.local,192.168.0.7,135.652835,0,100,"4_learning_rate=0.00097977,n_lags=20,num_hidde...",20,0.000980,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
b82cf_00115,0.780160,1.263130,True,None,None,10,e97b876c9c4d453dbc89468805ca85f6,2021-05-11_00-28-46,1620682126,22.238568,1766,MacBook-Pro-Polina.local,192.168.0.7,22.238568,0,10,"115_learning_rate=0.00036596,n_lags=20,num_hid...",20,0.000366,16
b82cf_00116,1.139772,1.367162,True,None,None,10,53c142b7b9d64296b223b82346c6d1f2,2021-05-11_00-28-44,1620682124,20.402098,1763,MacBook-Pro-Polina.local,192.168.0.7,20.402098,0,10,"116_learning_rate=0.00023846,n_lags=30,num_hid...",30,0.000238,8
b82cf_00117,0.000299,0.826584,True,None,None,40,8f34ca39e1f54677a6a2e6def368414f,2021-05-11_00-29-16,1620682156,38.893304,1781,MacBook-Pro-Polina.local,192.168.0.7,38.893304,0,40,"117_learning_rate=0.066142,n_lags=10,num_hidde...",10,0.066142,8


In [8]:
best_params

{'n_lags': 10, 'learning_rate': 0.05765758454779955, 'num_hidden_layers': 8}